In [1]:
'''

Предполагается, что датасет выглядит как-то так: 

<IN>Современное прочтение СТЭ не отрицает наличие ЭС и ее влияние на видообразование, но делает акцент на "мутационном факторе" изменчивости. В любом случае, благодарю за ваш ответ по существу. 🤝
<OUT>Все примеры мутационной изменчивости не столь очевидны (особенно с учетом отсутствия прямого соответствия ген-признак).  Всегда есть вероятность априорного существования соответствующего морфоза.
<END>
<IN>Просто тема сама рифмуется с соразмерным госфинансированием...
<OUT>ну что вам так финансирование далось... всем фундаментальным проектам не хватает финансирования, точнее, с финансированием происходит полный рандом. это нормально. вот будет конкретный прикладной проект — тогда и надо будет выбивать для него финансирование
<END>

В простом тесктовом файле...

'''

'\n\nПредполагается, что датасет выглядит как-то так: \n\n<IN>Современное прочтение СТЭ не отрицает наличие ЭС и ее влияние на видообразование, но делает акцент на "мутационном факторе" изменчивости. В любом случае, благодарю за ваш ответ по существу. 🤝\n<OUT>Все примеры мутационной изменчивости не столь очевидны (особенно с учетом отсутствия прямого соответствия ген-признак).  Всегда есть вероятность априорного существования соответствующего морфоза.\n<END>\n<IN>Просто тема сама рифмуется с соразмерным госфинансированием...\n<OUT>ну что вам так финансирование далось... всем фундаментальным проектам не хватает финансирования, точнее, с финансированием происходит полный рандом. это нормально. вот будет конкретный прикладной проект — тогда и надо будет выбивать для него финансирование\n<END>\n\nВ простом тесктовом файле... тип датасета что-то вроде QA или conversation\n\n'

In [17]:
#@title Загрузка исходной модели { vertical-output: true }
from transformers import AutoTokenizer, AutoModelForCausalLM
#@markdown **sberbank-ai/rugpt3medium_based_on_gpt2** - довольно "жирная" модель, нужен GPU c большим объемом памяти 
model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
#@markdown если нет такого, то можно **sberbank-ai/rugpt3small_based_on_gpt2** попробовать

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/sberbank-ai/rugpt3medium_based_on_gpt2/resolve/main/config.json from cache at E:\HF_HOME\transformers\c73d5ea1365be5d1d2cc9f3ecb5c6afd77fbd85274871869ff81afeb6e5bbfd5.c7f979c8d927f0e43eb584070478140e210e1b8830f862d45e9f3d39eda2ef5d
Model config GPT2Config {
  "_name_or_path": "sberbank-ai/rugpt3medium_based_on_gpt2",
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 2048,
  "n_special": 0,
  "output_past": true,
  "predict_special_tokens": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "s

In [18]:
#@title Модификация токенайзера  { vertical-output: true }
#@markdown добавляем несколько токенов нашей разметки
special_tokens_dict = {'additional_special_tokens': ['<IN>','<OUT>','<END>','|PAD|']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))
print(tokenizer.all_special_tokens)
tokenizer.pad_token = '|PAD|'

Assigning ['<IN>', '<OUT>', '<END>', '|PAD|'] to the additional_special_tokens key of the tokenizer


['<|endoftext|>', '<IN>', '<OUT>', '<END>', '|PAD|']


In [4]:
cd D:\PROJECTS\Telegram data

D:\PROJECTS\Telegram data


In [11]:
#@title Загрузка датасета  { vertical-output: true }

#@markdown теоретически, это устревший формат датасета, но на практике

#@markdown он дает равномерную загрузку графической памяти и позволяет 

#@markdown использовать больший батч.

from transformers import TextDataset,DataCollatorForLanguageModeling

train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path='all.txt',
          block_size=128)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

D:\Conda\envs\Work\lib\site-packages\transformers\data\datasets\language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Loading features from cached file cached_lm_GPT2TokenizerFast_128_all.txt [took 0.240 s]


In [ ]:
#@title Загрузка датасета  { vertical-output: true }
#@markdown Это вроде как правильный способ, но мне не понравился...
#@markdown Можно не запускать - не используется.

from datasets import load_dataset

#data_files = {'train': 'train.txt', 'test': 'test.txt'}
ds = load_dataset('text', data_files = 'all.txt')

def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized_dataset = ds.map(tokenize_function, batched=True)

In [6]:
#@title Параметры обучения { vertical-output: true } 

from transformers import Trainer, TrainingArguments
import torch

#@markdown если не лезет в память, можно уменьшать **per_device_train_batch_size**
training_args = TrainingArguments(
    output_dir="./AGI-bot", #The output directory
    #evaluation_strategy="epoch",
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size= 25, # batch size for training
    #per_device_eval_batch_size=50,  # batch size for evaluation
    eval_steps = 500, # Number of update steps between two evaluations.
    save_steps=1000, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    #train_dataset=tokenized_dataset['train'],
    train_dataset=train_dataset,
    #eval_dataset=test_dataset,
    #compute_metrics=compute_metrics,
)

torch.cuda.empty_cache()

In [ ]:
#@title Обучение { vertical-output: true } 
trainer.train()

D:\Conda\envs\Work\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 28687
  Num Epochs = 3
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 3588


Step,Training Loss
500,3.829200
1000,3.496900
1500,3.302200
2000,3.188900
2500,3.116200
3000,2.981500
3500,2.965200


Saving model checkpoint to ./AGI-bot\checkpoint-800
Configuration saved in ./AGI-bot\checkpoint-800\config.json
Model weights saved in ./AGI-bot\checkpoint-800\pytorch_model.bin
Saving model checkpoint to ./AGI-bot\checkpoint-1600
Configuration saved in ./AGI-bot\checkpoint-1600\config.json
Model weights saved in ./AGI-bot\checkpoint-1600\pytorch_model.bin
Saving model checkpoint to ./AGI-bot\checkpoint-2400
Configuration saved in ./AGI-bot\checkpoint-2400\config.json
Model weights saved in ./AGI-bot\checkpoint-2400\pytorch_model.bin
Saving model checkpoint to ./AGI-bot\checkpoint-3200
Configuration saved in ./AGI-bot\checkpoint-3200\config.json
Model weights saved in ./AGI-bot\checkpoint-3200\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3588, training_loss=3.2611268042454884, metrics={'train_runtime': 1695.4189, 'train_samples_per_second': 50.761, 'train_steps_per_second': 2.116, 'total_flos': 1.998122761794355e+16, 'train_loss': 3.2611268042454884, 'epoch': 3.0})

In [26]:
#@title специальные токены { vertical-output: true }
T_OUT = tokenizer.encode('<OUT>')[0]
T_END = tokenizer.encode('<END>')[0]
T_PAD = tokenizer.encode('|PAD|')[0]

T_OUT, T_END, T_PAD

(50259, 50260, 50261)

In [30]:
#@title Инференс (тестирование) { vertical-output: true }
#@markdown кавычки не убирать!!!
text = "Здравствуй железяка разумная! Как у тебя сегодня дела?" #@param {type:"raw"}
model.cuda()

text = f"<IN>{text}\n<OUT>"
inpt = tokenizer.encode(text, return_tensors="pt")
inpt= inpt.cuda()

out = model.generate(inpt,  max_length=len(inpt[0])+300, do_sample=True, top_k=5, top_p=0.95, temperature=1, eos_token_id=T_END, pad_token_id=T_PAD)

out_tokens = torch.where(out[0]==T_OUT)
last_repl = out[0][out_tokens[0][-1]+1:-1]
repl = tokenizer.decode(last_repl)

print(repl)

Да вот, решил с утра пораньше заняться творчеством.



In [16]:
# сохранение модели. В таком виде ее можно загрузить на huggingface
#@title Сохраняем модель и токенайзер { vertical-output: true } 

#@markdown в таком виде уже можно закинуть на https://huggingface.co/
#@markdown ,но это уже отдельная тема и не в блокноте 

model.save_pretrained('AGIRussia')
tokenizer.save_pretrained('AGIRussia')

Configuration saved in AGIRussia\config.json
Model weights saved in AGIRussia\pytorch_model.bin
tokenizer config file saved in AGIRussia\tokenizer_config.json
Special tokens file saved in AGIRussia\special_tokens_map.json


('AGIRussia\\tokenizer_config.json',
 'AGIRussia\\special_tokens_map.json',
 'AGIRussia\\vocab.json',
 'AGIRussia\\merges.txt',
 'AGIRussia\\added_tokens.json',
 'AGIRussia\\tokenizer.json')

In [24]:
#@title А вот так потом с huggingface достаем и используем { vertical-output: true } 
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "Nehc/AGIRussia" #"models\AGIRussia"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

loading file https://huggingface.co/Nehc/AGIRussia/resolve/main/vocab.json from cache at E:\HF_HOME\transformers\6830227586c746675d384959b1d44c547d38042e6276857d01a41fbb241e80b5.b488da21e063aa1d3ff664d686226d61ee451181ff9213f3da996b42d42f384b
loading file https://huggingface.co/Nehc/AGIRussia/resolve/main/merges.txt from cache at E:\HF_HOME\transformers\8e8ebfbf1549d9b778ce404310b754fab1aa6a0a9b1f6881fb2321d13b5fe081.db5526f107eff4fd6c62a9a36a55cd29713e2ebbf3ffa34a9dc838d05f93e811
loading file https://huggingface.co/Nehc/AGIRussia/resolve/main/tokenizer.json from cache at E:\HF_HOME\transformers\1af74ec5a2982000dc0f016692ffcdba3d1ab47a8fc3562a4c2ae95609966cef.af72d954e5ea573a8215b12e8a8b37788a01cedbc7682b377a3bef123ea3ba33
loading file https://huggingface.co/Nehc/AGIRussia/resolve/main/added_tokens.json from cache at E:\HF_HOME\transformers\941aae9037065c7692ec4bfcfc42d6af925be53e9c48dd0e064663b7ba2a0ece.4ee0ab48916932a0b9c86a3fb9bea5a7179927323c5e6e72b8d7e37815b30935
loading file http